In [1]:
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
model_conv = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [3]:
model_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator = data_generator.flow_from_directory(
    'PetImages',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1190 images belonging to 2 classes.


In [5]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

Found 1399 images belonging to 2 classes.


In [6]:
from tensorflow.keras import models, layers, optimizers

In [7]:
model_conv.trainable = True
set_trainble = False
for layer in model_conv.layers:
    if layer.name == 'block5_conv1':
        set_trainble = True
    if set_trainble == True:
        layer.trainable = True
    else:
        layer.trainable = False

In [8]:
model = models.Sequential()
model.add(model_conv)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [12]:
model.fit(generator, steps_per_epoch=50, epochs=3, validation_data=generator_test, validation_steps=30)

Epoch 1/3
10/10 [==============================] - 23s 2s/step - loss: 0.4006 - accuracy: 0.8450 - val_loss: 1.0321 - val_accuracy: 0.5000
Epoch 2/3
10/10 [==============================] - 26s 3s/step - loss: 0.3882 - accuracy: 0.8200 - val_loss: 0.7449 - val_accuracy: 0.5033
Epoch 3/3
10/10 [==============================] - 25s 3s/step - loss: 0.3285 - accuracy: 0.8300 - val_loss: 1.1676 - val_accuracy: 0.4967


In [11]:
# MasoudKaviani.ir